Build dummy regressors

Based on earlier analyses from 12a-dummy_pred_actual. Here also derive the concordance. This is used for Lx=L0.

In [72]:
import pickle
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
from ceres_infer.analyses import getConcordance
from tqdm import tqdm

In [89]:
from sklearn.dummy import DummyRegressor
def getDummyInfer(y):
    dummy_clf = DummyRegressor(quantile=0.5)
    dummy_clf.fit([0]*y.shape[0], y)
    return dummy_clf.predict([0]*y.shape[0])

def getDummyInferFromTrain(y, y_tr):
    return [y_pred_tr[y.name][0]]*len(y)

# for genome wide

In [ ]:
dir_in_Lx = '../out/20.0518 Lx/L100only_reg_rf_boruta_all/' # just using this to get the actual y values

y_compr_tr = pickle.load(open(os.path.join(dir_in_Lx, 'anlyz', 'y_compr_tr.pkl'), 'rb'))
y_compr_te = pickle.load(open(os.path.join(dir_in_Lx, 'anlyz', 'y_compr_te.pkl'), 'rb'))

In [90]:
tqdm.pandas()
y_pred_tr = y_compr_tr['actual'].progress_apply(getDummyInfer, axis=0)
y_pred_te = y_compr_te['actual'].progress_apply(getDummyInferFromTrain, y_tr=y_pred_tr, axis=0)

/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 18228/18228 [00:02<00:00, 7976.82it/s] 


In [91]:
outdir_concord = '../out/20.0909 Lx/L0only_reg_dummy/anlyz/concordance'
if not os.path.exists(outdir_concord):
    os.makedirs(outdir_concord)
df_conc_tr = pd.DataFrame()
df_conc_te = pd.DataFrame()
for gene in tqdm(y_pred_tr.columns.values):
    df = pd.DataFrame({'y_actual': y_compr_tr['actual'].loc[:,y_compr_tr['actual'].columns == gene].values.squeeze(),
                       'y_pred': y_pred_tr.loc[:,y_pred_tr.columns == gene].values.squeeze(),
                      })
    tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df)}])
    df_conc_tr = pd.concat([df_conc_tr, tmp])

    df = pd.DataFrame({'y_actual': y_compr_te['actual'].loc[:,y_compr_te['actual'].columns == gene].values.squeeze(),
                       'y_pred': y_pred_te.loc[:,y_pred_te.columns == gene].values.squeeze(),
                      })
    tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df)}])
    df_conc_te = pd.concat([df_conc_te, tmp])
    
df_conc_tr.to_csv('%s/concordance_tr.csv' % outdir_concord, index=False)
df_conc_te.to_csv('%s/concordance_te.csv' % outdir_concord, index=False)

plt.figure()
ax = sns.distplot(df_conc_tr.concordance)
ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
plt.savefig("%s/concordance_tr.pdf" % outdir_concord)
plt.close()

plt.figure()
ax = sns.distplot(df_conc_te.concordance)
ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
plt.savefig("%s/concordance_te.pdf" % outdir_concord)
plt.close()
    

100%|██████████| 18228/18228 [05:01<00:00, 60.46it/s]


# for selective essential

In [119]:
dir_in_Lx = '../out/20.0518 Lx/L100only_reg_rf_boruta/' # just using this to get the actual y values

y_compr_tr = pickle.load(open(os.path.join(dir_in_Lx, 'anlyz', 'y_compr_tr.pkl'), 'rb'))
y_compr_te = pickle.load(open(os.path.join(dir_in_Lx, 'anlyz', 'y_compr_te.pkl'), 'rb'))

In [120]:
tqdm.pandas()
y_pred_tr = y_compr_tr['actual'].progress_apply(getDummyInfer, axis=0)
y_pred_te = y_compr_te['actual'].progress_apply(getDummyInferFromTrain, y_tr=y_pred_tr, axis=0)

/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 535/535 [00:00<00:00, 6988.09it/s]


In [122]:
outdir_concord = '../out/20.0909 Lx/L0only_reg_dummy/anlyz/concordance'
if not os.path.exists(outdir_concord):
    os.makedirs(outdir_concord)
df_conc_tr = pd.DataFrame()
df_conc_te = pd.DataFrame()
for gene in tqdm(y_pred_tr.columns.values):
    df = pd.DataFrame({'y_actual': y_compr_tr['actual'].loc[:,y_compr_tr['actual'].columns == gene].values.squeeze(),
                       'y_pred': y_pred_tr.loc[:,y_pred_tr.columns == gene].values.squeeze(),
                      })
    tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df)}])
    df_conc_tr = pd.concat([df_conc_tr, tmp])

    df = pd.DataFrame({'y_actual': y_compr_te['actual'].loc[:,y_compr_te['actual'].columns == gene].values.squeeze(),
                       'y_pred': y_pred_te.loc[:,y_pred_te.columns == gene].values.squeeze(),
                      })
    tmp = pd.DataFrame([{'gene': gene, 'concordance': getConcordance(df)}])
    df_conc_te = pd.concat([df_conc_te, tmp])
    
df_conc_tr.to_csv('%s/concordance_tr.csv' % outdir_concord, index=False)
df_conc_te.to_csv('%s/concordance_te.csv' % outdir_concord, index=False)

plt.figure()
ax = sns.distplot(df_conc_tr.concordance)
ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
plt.savefig("%s/concordance_tr.pdf" % outdir_concord)
plt.close()

plt.figure()
ax = sns.distplot(df_conc_te.concordance)
ax.set(xlim=[0, 1.05], xlabel='Concordance', title='Concordance between actual and predicted')
plt.savefig("%s/concordance_te.pdf" % outdir_concord)
plt.close()
    

100%|██████████| 535/535 [00:05<00:00, 99.87it/s] 


In [118]:
# # amend the L100only_reg_rf_boruta results with the y_compr_tr and y_compr_te
# from ceres_infer.analyses import yComprHeatmap, constructYCompr
# outdir_anlyz = '../out/20.0518 Lx/L100only_reg_rf_boruta/anlyz/'
# model_results = pd.read_csv('../out/20.0518 Lx/L100only_reg_rf_boruta/model_results.csv', header=0)
# outdir_modtmp = '../out/20.0518 Lx/L100only_reg_rf_boruta/model_perf/'

# #------- Y predictions comparisons -------
# # create dir
# outdir_ycompr = '%s/heatmaps/' % outdir_anlyz
# if not os.path.exists(outdir_ycompr):
#     os.makedirs(outdir_ycompr)

# genes2analyz = model_results.target.unique()
# y_compr_fnames = glob.glob(os.path.join(outdir_modtmp, 'y_compr_*.pkl'))

# if (len(y_compr_fnames) > 0) and (len(genes2analyz) > 0):
#     # for train
#     df_y_actual, df_y_pred = constructYCompr(genes2analyz, 'tr', outdir_modtmp)
#     pickle.dump({'actual': df_y_actual, 'predicted': df_y_pred}, open('%s/y_compr_tr.pkl' % outdir_anlyz, 'wb'))
#     yComprHeatmap(df_y_actual, df_y_pred, 'tr', outdir_ycompr)

#     # for test
#     df_y_actual, df_y_pred = constructYCompr(genes2analyz, 'te', outdir_modtmp)
#     pickle.dump({'actual': df_y_actual, 'predicted': df_y_pred}, open('%s/y_compr_te.pkl' % outdir_anlyz, 'wb'))
#     yComprHeatmap(df_y_actual, df_y_pred, 'te', outdir_ycompr)